In [32]:
import spotipy
import urllib.request
import random
import requests
import nltk
import webbrowser
import json
from nltk import pos_tag
import spotipy.util as util

In [2]:
with open("news_key.txt", "r") as file:
    key = file.read()

In [3]:
url = "https://newsapi.org/v2/top-headlines?country=us&category=entertainment&apiKey=" + key

In [4]:
url

'https://newsapi.org/v2/top-headlines?country=us&category=entertainment&apiKey=19e7b8a1be8f48a790f4655dde3660e4'

In [5]:
# requesting the information
request = urllib.request.Request(url)
# the information is coming back to me
response = urllib.request.urlopen(request)

In [6]:
response

In [11]:
# to get the data out of the response object
headlines = json.loads(response.read())

In [12]:
headlines

{'status': 'ok',
 'totalResults': 68,
 'articles': [{'title': 'The Best Red Carpet Looks At Beyoncé\'s "Renaissance" Film Premiere - Essence',
   'author': 'Kerane Marcellus',
   'source': {'id': None, 'name': 'Essence'},
   'publishedAt': '2023-11-27T13:49:00Z',
   'url': 'http://www.essence.com/gallery/renaissance-film-beyonce-premiere-best-red-carpet-looks/'},
  {'title': 'Taylor Swift poses for photo with family of fan who died at Eras Tour show - Page Six',
   'author': 'Riley Cardoza',
   'source': {'id': None, 'name': 'Page Six'},
   'publishedAt': '2023-11-27T13:13:00Z',
   'url': 'https://pagesix.com/2023/11/27/entertainment/taylor-swift-meets-family-of-fan-who-died-at-eras-tour-show/'},
  {'title': 'Craziest WWE mismatches: WWE Playlist - WWE',
   'author': None,
   'source': {'id': None, 'name': 'YouTube'},
   'publishedAt': '2023-11-27T13:00:00Z',
   'url': 'https://www.youtube.com/watch?v=-wLdd4l4YXE'},
  {'title': "Irish author Paul Lynch wins Booker Prize for 'Prophet So

In [13]:
headlines['articles'][0]['title']

'The Best Red Carpet Looks At Beyoncé\'s "Renaissance" Film Premiere - Essence'

In [14]:
article_titles = [article['title'] for article in headlines['articles']]

In [15]:
article_titles

['The Best Red Carpet Looks At Beyoncé\'s "Renaissance" Film Premiere - Essence',
 'Taylor Swift poses for photo with family of fan who died at Eras Tour show - Page Six',
 'Craziest WWE mismatches: WWE Playlist - WWE',
 "Irish author Paul Lynch wins Booker Prize for 'Prophet Song' - DW (English)",
 "John Travolta nearly died after his plane had 'total electrical failure' - New York Post ",
 'Historians absolutely hate Ridley Scott Napoleon movie - Insider',
 "'Reindeer in Here': Why a dad made an 'Elf on the Shelf' alternative - USA TODAY",
 'Last chance: Hulu and Disney Plus Cyber Monday deal combo $2.99/month - Space.com',
 'Country music couples Tim McGraw, Faith Hill and Garth Brooks, Trisha Yearwood share tips to lasting marriages - Fox News',
 "Kim Kardashian's fake 'nipple bra' to ensure eye-popping view through clothing sold out online - Fox News",
 'Bradley Cooper supports Brad Pitt, Brooke Shields as real-life Hollywood hero - Fox News',
 'Tiny Desk Korea: Tomorrow X Togethe

In [21]:
tagged = pos_tag(article_titles[0].split())
tagged

[('The', 'DT'),
 ('Best', 'NNP'),
 ('Red', 'NNP'),
 ('Carpet', 'NNP'),
 ('Looks', 'NNP'),
 ('At', 'IN'),
 ("Beyoncé's", 'NNP'),
 ('"Renaissance"', 'NNP'),
 ('Film', 'NNP'),
 ('Premiere', 'NNP'),
 ('-', ':'),
 ('Essence', 'NN')]

In [24]:
for tag in tagged:
    print(tag)

('The', 'DT')
('Best', 'NNP')
('Red', 'NNP')
('Carpet', 'NNP')
('Looks', 'NNP')
('At', 'IN')
("Beyoncé's", 'NNP')
('"Renaissance"', 'NNP')
('Film', 'NNP')
('Premiere', 'NNP')
('-', ':')
('Essence', 'NN')


In [27]:
important_words = []
for article in article_titles:
    tagged = pos_tag(article.split())
    # only select the proper nouns
    important_words.extend([word for word,tag in tagged if tag == 'NNP'])

important_words

['Best',
 'Red',
 'Carpet',
 'Looks',
 "Beyoncé's",
 '"Renaissance"',
 'Film',
 'Premiere',
 'Taylor',
 'Swift',
 'Eras',
 'Tour',
 'Craziest',
 'WWE',
 'WWE',
 'Playlist',
 'Paul',
 'Lynch',
 'Booker',
 'Prize',
 "'Prophet",
 "Song'",
 'DW',
 'John',
 'Travolta',
 'New',
 'York',
 'Post',
 'Ridley',
 'Scott',
 'Napoleon',
 "Here':",
 "Shelf'",
 'Hulu',
 'Disney',
 'Plus',
 'Cyber',
 'Monday',
 '$2.99/month',
 'Country',
 'Tim',
 'McGraw,',
 'Faith',
 'Hill',
 'Garth',
 'Brooks,',
 'Trisha',
 'Yearwood',
 'Fox',
 'News',
 'Kim',
 "Kardashian's",
 'Bradley',
 'Cooper',
 'Brad',
 'Pitt,',
 'Brooke',
 'Shields',
 'Hollywood',
 'Tiny',
 'Desk',
 'Korea:',
 'Tomorrow',
 'X',
 'Together',
 'Music',
 'Bill',
 'Engvall',
 'California',
 'Utah',
 'MegaUpload',
 'Founder',
 'Kim',
 'Dotcom',
 'Biopic',
 'Treatment',
 'Hollywood',
 'Reporter',
 'Kantara',
 'A',
 'Legend',
 'Chapter-1',
 'First',
 'Look:',
 'Large!',
 'Rumor',
 'Roundup:',
 'CM',
 'Punk’s',
 'WWE',
 'Drew',
 'McIntyre',
 'WWE',
 '

In [29]:
with open("spotify_keys.json", "r") as file:
    api_tokens = json.load(file)

In [30]:
client_id = api_tokens["client_id"]
client_secret = api_tokens["client_secret"]
redirectURI = api_tokens["redirect"]
username = api_tokens["username"]

In [ ]:
scope = 'user-read-private user-read-playback-state user-modify-playback-state playlist-modify-public user-library-read'
token = util.prompt_for_user_token(username, scope, client_id=client_id,
                           client_secret=client_secret,
                           redirect_uri=redirectURI)

In [ ]:
token

In [37]:
sp = spotipy.Spotify(auth=token)